<h1> Artificial and Computational Intelligence Assignment 2 </h1>

<h3> Gaming: Catch-Up with numbers </h3>

| S.No. | BITS ID | Name | Contribution |
|------ |----------|----------|----------|
| 1    | 2024AA05046    | NAGARJUN K S | 100 %  | 
| 2    | 2024AA05047    | JEETKUMAR VIJAYKUMAR PATEL  | 100 % | 
| 3    | 2024AA05048    | SAHAJ ROHILLA | 100 % |
| 4    | 2024AA05049    | POORVA AGARWAL | 100 % | 
| 5    | 2024AA05050    | DARJI JITARKKUMAR MANOJKUMAR| 100 % |

In [15]:
#Importing nessesary libraries
import itertools  # Used to generate all possible subsets of available numbers
import random  # Used for random selections
import time  # Used to introduce delays for better game interaction

"""Finds the smallest subset of available numbers whose sum is at least target_sum.
   This helps in making efficient moves instead of generating all possible subsets.""" 
def get_valid_subsets(available_numbers, target_sum):
    sorted_numbers = sorted(available_numbers, reverse=True)  # Sort in descending order
    best_subset = set()
    current_sum = 0

    """ 
    Iterate over sorted numbers and pick the smallest set that meets/exceeds target_sum.
    This ensures that we find the optimal move efficiently.
    """
    for num in sorted_numbers:
        best_subset.add(num)
        current_sum += num
        if current_sum >= target_sum:  # Stop when we reach or exceed the target
            break

    return [best_subset] if best_subset else [set()]

"""Implements the Minimax algorithm to evaluate the best possible move for the player.
   Maximizing for P1, minimizing for P2.""" 
def minimax(available_numbers, p1_sum, opponent_sum, is_maximizing, depth):
    # Base case: If no numbers left, depth reached, or condition met, return evaluation.
    # The evaluation function favors the player with the highest sum.
    if not available_numbers or depth == 0 or (p1_sum > opponent_sum and not is_maximizing):
        return p1_sum - opponent_sum if is_maximizing else opponent_sum - p1_sum  
    
    target_sum = max(p1_sum, opponent_sum)  # Player must match or exceed the opponent's score
    valid_moves = get_valid_subsets(available_numbers, target_sum)

    if is_maximizing:
        best_score = float('-inf')

        #Iterate over all valid moves and choose the one that maximizes the score.
        for move in valid_moves:
            new_available = available_numbers - move
            score = minimax(new_available, p1_sum + sum(move), opponent_sum, False, depth-1)
            best_score = max(best_score, score)
        
        return best_score
    else:
        best_score = float('inf')

        #Iterate over all valid moves and choose the one that minimizes the opponent's score.
        for move in valid_moves:
            new_available = available_numbers - move
            score = minimax(new_available, p1_sum, opponent_sum + sum(move), True, depth-1)
            best_score = min(best_score, score)
        
        return best_score

"""Determines the best possible move using the Minimax algorithm.
   Selects a subset of available numbers that leads to the best outcome.""" 
def best_move(available_numbers, p1_sum, opponent_sum, depth=None):
    if depth is None:
        depth = min(2, max(1, 4 - (len(available_numbers) // 15)))  # Adjust depth dynamically
    
    target_sum = max(p1_sum, opponent_sum)
    valid_moves = get_valid_subsets(available_numbers, target_sum)

    if not valid_moves:
        return {min(available_numbers)} if available_numbers else None  # If no move possible, pick the smallest

    best_choice = None
    best_score = float('-inf')

    # Iterate over all valid moves and find the best scoring move using Minimax.
    for move in valid_moves:
        new_available = available_numbers - move
        score = minimax(new_available, p1_sum + sum(move), opponent_sum, False, depth)
        if score > best_score:
            best_score = score
            best_choice = move

    return best_choice

"""Main function to run the Catch-Up game interactively.
   Handles player turns, input validation, and displays the game state.""" 
def play_game(n):
    available_numbers = set(range(1, n+1))  # Set of available numbers from 1 to n
    p1_sum = 0  # Player 1's total score
    p2_sum = 0  # Player 2's total score
    p1_first_turn = True  # Track if it's P1's first move

    print("\n🎮 Welcome to Catch-Up! You are player P1 and you play against player P2. 🎮")
    print("\n📜 Game Rules:")
    print("\t• On your first turn, you (P1) can only pick ONE number! After that, you may pick multiple numbers.")
    print("\t• Your (P1) sum must equal or exceed P2's last sum.\n")

    while available_numbers:
        print(f"\n🎲 Available numbers: {sorted(available_numbers)}")

        #Check if there are valid moves left for P1. If no valid move exists, P2 wins by default.
        valid_moves_exist = any(
            sum(subset) >= p2_sum 
            for i in range(1, len(available_numbers) + 1) 
            for subset in itertools.combinations(available_numbers, i)
        )
        if not valid_moves_exist:
            print("🚫 No valid moves left for you(P1)! P2 has won!")
            break

        while True:
            try:
                player_choice = set(map(int, input("\n📝 Your(P1) turn! Choose numbers separated by spaces: ").split()))
                
                # Validate if all selected numbers exist in the available list.
                if not player_choice.issubset(available_numbers):
                    print("⚠️ Please choose the values in the list.")
                    continue
                
                # First turn validation: Only allow P1 to pick 1 number on the first move.
                if p1_first_turn and len(player_choice) > 1:
                    print("⚠️ On your(P1) first turn, you can only choose ONE number. Try again.")
                    continue

                # Validate sum condition: Selected numbers must meet or exceed P2's last total.
                if sum(player_choice) + p1_sum >= p2_sum:
                    available_numbers -= player_choice
                    p1_sum += sum(player_choice)
                    print(f"✅ You(P1) selected: {player_choice} | Total: {p1_sum}")
                    p1_first_turn = False  # After first move, allow multiple selections
                    time.sleep(1)
                    break
                else:
                    print("⚠️ Invalid move! Your(P1) total must be equal to or greater than P2's last total. Try again.")
            except ValueError:
                print("⚠️ Invalid input! Please enter numbers separated by spaces.")

        if not available_numbers:
            break  # End game if all numbers are chosen

        print("\n🤖 P2 is thinking...")
        time.sleep(0.5)
        p2_choice = best_move(available_numbers, p2_sum, p1_sum)
        if p2_choice is None or not p2_choice:
            if available_numbers:
                p2_choice = {min(available_numbers)}
            else:
                print("🤖 P2 has no valid moves left! Game over!")
                break

        available_numbers -= p2_choice
        p2_sum += sum(p2_choice)
        print(f"🤖 P2 chooses: {p2_choice} | Total: {p2_sum}")
        time.sleep(1)

    # Display Final Scores and determine the winner.
    print("\n🎯 Final Scores:")
    print(f"👤 You(P1): {p1_sum} | 🤖 P2: {p2_sum}")
    if p1_sum > p2_sum:
        print("\n🎉 Congratulations! You win! 🏆")
    elif p1_sum < p2_sum:
        print("\n😔 P2 wins! Better luck next time! 🤖")
    else:
        print("\n🤝 It's a tie! Well played!")

# Get user input for n
n = int(input("\n🔢 Enter the value of n: "))
play_game(n)


🔢 Enter the value of n: 15

🎮 Welcome to Catch-Up! You are player P1 and you play against player P2. 🎮

📜 Game Rules:
	• On your first turn, you (P1) can only pick ONE number! After that, you may pick multiple numbers.
	• Your (P1) sum must equal or exceed P2's last sum.


🎲 Available numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

📝 Your(P1) turn! Choose numbers separated by spaces: 9
✅ You(P1) selected: {9} | Total: 9

🤖 P2 is thinking...
🤖 P2 chooses: {15} | Total: 15

🎲 Available numbers: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14]

📝 Your(P1) turn! Choose numbers separated by spaces: 14
✅ You(P1) selected: {14} | Total: 23

🤖 P2 is thinking...
🤖 P2 chooses: {12, 13} | Total: 40

🎲 Available numbers: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11]

📝 Your(P1) turn! Choose numbers separated by spaces: 10 11 8 7 6 5 4
✅ You(P1) selected: {4, 5, 6, 7, 8, 10, 11} | Total: 74

🤖 P2 is thinking...
🤖 P2 chooses: {1, 2, 3} | Total: 46

🎯 Final Scores:
👤 You(P1): 74 | 🤖 P2: 46

🎉 Congratul